In [1]:
#오차역전법

In [3]:
# 계산 그래프에서 사용 되는 곱셈 노드와 덧셈노드를 구현
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self,x,y):
    self.x = x
    self.y = y
    out = x*y
    return out

  def backward(self,dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx,dy

  # 덧셈 노드
class AddLayer:
  def __init__(self):
    pass
  def forward(self,x,y):
    out = x+y
    return out

  def backward(self,dout):
    dx = dout * 1
    dy = dout * 1
    return dx,dy
    

In [6]:
apple = 100
apple_num = 2 
tax = 1.1

# 노드 정의
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forwarding
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)

# backward
dprice = 1 # 가격의증가분/가격의증가분
dapple_price,dtax = mul_tax_layer.backward(dprice)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)


In [9]:
print(int(price))
print(dapple_price,dtax)
print(dapple,dapple_num)


220
1.1 200
2.2 110.00000000000001


In [10]:
# affine 
class Affine:
  def __init__(self,w,b):
    self.w = w
    self.b = b
    self.x = None
    self.dw = None
    self.db = None
  def forward(self,x):
    self.x=x
    out = np.dot(x,self.w)+self.b
    return out
  def backward(self,dout):
    dx = np.dot(dout,self.w.T)
    self.dw = np.dot(self.x.T,dout)
    self.db = np.sum(dout,axis = 0)
    return dx

In [11]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None
    self.t = None

  def forward(self,x,t):
    self.t =t
    self.y = softmax(x)
    self.loss = cross_entropy_error(t,y)
    return self.loss

  def backward(self,dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y-self.t)  / batch_size
    return dx
    


In [ ]:
# 오차 역전파법을 이용한 2층짜리 network
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    # 가중치 초기화 
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    # 계층 생성
    self.layers = OrderedDict() # 순서가 있는 딕셔너리
    self.layers['Affine1'] = Affine(self.params['w1'],self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['w2'],self.params['b2'])
    
    self.lastlayer = SoftmaxWithLoss()

  def predict(self,x):
    for layer in self.layers.values():
      x = layer.forward(x)
    
    return x

  def loss(self,x,t):
    y = self.predict(x)
    return self.lastlayer.forward(y,t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y==t) / float(x.shape[0])
    return accuracy

  # x : 입력 데이터, t : 정답 레이블
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)
    
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    
    return grads

  def gradient(self,x,t):
    # 순전파
    self.loss(x,t)

    # 역전파
    dout = 1 
    dout = self.lastlayer.backward(dout)

    layers = list(self.layers.values())
    layers.revers()

    for layer in layers:
      dout = layer.backward(dout)

      grads = {}
      grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
      grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
      grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
      grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    
    return grads
